In [ ]:
# 띄어쓰기 ngram
# q = 'ABCDE..'
# i = ^
# r = ''

# 2. gram
# i가 0일때 q의 index 0 이 나오는지 확인하고
# 해당 맞는 애들을 넣는다

In [ ]:
from os import listdir

def fileids(path):
    return list(map(lambda f:path + ('' if path[-1] == '/' else '/')+f, listdir(path)))

In [ ]:
# 띄어쓰기
def ngram(s, n=2, t=True): # t=True;어절, False;음절
    result = []
    
    if not t:
        s = list(s)
        
    for i in range(len(s)-(n-1)):
        result.append(tuple(s[i:i+n])) # 튜플로 수정
        
    return result

In [ ]:
def findKey(gram, key):
    k = tuple(key)
    return list(filter(lambda g:g[:len(k)] == k, gram.keys()))

In [ ]:
def autoSpacing(q, n=2):
    i = 0
    while i < len(q)-(n-1):
        i += 1

    return r

# autoSpacing()

In [ ]:
import re

# gram = {2:[],3,4,5}
corpus = list()
for f in fileids('news'):
    with open(f, 'r', encoding='utf8') as fp:
        corpus.append(
            re.sub(r'^\s+|\s+$', '',
                   re.sub(r'\s+', ' ',
                          re.sub(r'\sCopyright.+', ' ',
                                 re.sub(r'[\xa0-\xff]', ' ', fp.read())))))

In [ ]:
from nltk.tokenize import sent_tokenize

gram = {2:list(), 3:list(), 4:[], 5:[]}
for c in corpus:
    for s in sent_tokenize(c):
        gram[2].extend(ngram(s, 2, False))
        gram[3].extend(ngram(s, 3, False))
        gram[4].extend(ngram(s, 4, False))
        gram[5].extend(ngram(s, 5, False))

In [ ]:
from collections import Counter

gram = {2:Counter(gram[2]), 3:Counter(gram[3]), 4:Counter(gram[4]), 5:Counter(gram[5])}

In [ ]:
def autoSpacing(q, n=2):
    i = 0
    r = list()

    r.append(q[i:i+(n-1)])

    while i < len(q)-(n-1):
        k = r[-(n-1):]
        keys = findKey(gram[n], k)
        candidates = {j:gram[n][j] for j in keys}
        best = sorted(candidates.items(), key = lambda r:r[1], revese=True)[0]
        if best[0][-1] == ' ':
            r.append(' ')
        else:
            r.append(q[i+(n-1)])
            i += 1

    return ''.join(r)

autoSpacing(sent_tokenize(corpus[0])[0].replace(' ',''))

In [ ]:
findKey(ngram[2], '초')

In [ ]:
# Branch Entropy
# -sum(p*log(p))
# 통계적 지식 없이 어감 분류

tokens = Counter('\n'.join(corpus).split())

In [ ]:
from math import log
from collections import defaultdict

def stemming(q):
    result = list()
    for i in range(len(q)):
        candidates = defaultdict(lambda:0)
        given = sum([tokens[k] for k in tokens.keys() if re.match(q[:i+1], k)])
        p = [tokens[k]/given for k in tokens.keys() if re.match(q[:i+2], k)]
        result.append(-sum([cp*log(cp) for cp in p]))

    return result

stemming('실행하고')

# 이런거 할 때는 작은 데이터셋으로 먼저 해보고
# 큰 데이터 셋으로 진행하는게 좋아보인다

In [ ]:
# perplexity

# 응집력을 뜻함
# clustering 할 때 perplexity로 평가를 한다
# perplexity가 크면 응집력이 쎄다
# 여기서 값이 많이 올라가는 부분을 찾아서 오류를 검출 가능하다

In [ ]:
# 전체 음절 ㅠp(xt+1|x1,...,xt)
# (4)   ==> sqrt(1/(p(x2|x1)*p(x3|x1,x2)*p(x4|x1,x2,x3)), 4) (1/4 제곱근)
#                   p(x1,x2) / x1 * p(x1,x2,x3) / x1,x2 * p(x1,x2,x3,x4) / x1,x2,x3 = p(x1,x2,x3,x4) / p(x1) = sqrt(freq(x1,x2,x3,x4) / freq(x1))
# 응집력에 대한 값 = cohesion score

In [ ]:
from math import sqrt

def cohension(t):
    t1 = t[:1]
    freq_t1 = sum([tokens[x] for x in list(filter(lambda k: re.match(t1, k), tokens.keys()))])
    print(t1, freq_t1)

    threshold = 0.0

    for i in range(1, len(t)):
        t2 = t[:i+1]
        freq_t2 = sum([tokens[x] for x in list(filter(lambda k: re.match(t2, k), tokens.keys()))])
        score = (freq_t2/freq_t1)**(1/len(t2))
        # print(t2, freq_t1/freq_t2)
        if score < threshold:
            print(i)
        threshold = score
    return (t,) if i == len(t) else (t[:i], t[i:])

cohension('금융감독원')

In [35]:
# tokenizing
# sent_tokenize, word_tokenize, regex_tokenize, TweetTokinize <= 언어적x, 구두점 (문장기호)로 잘라내고 나눠냄
# 한국어 특징 반영하려함 => 형태소 분석기를 사용하여 혹은 명사 추출기, 품사 부착기를 사용하여 반영 시도
# 언어적 특징 이용 => ngram
# 통계적 방법 => branch-entropy, perplexity => 응집력에 대한 것들

In [36]:
# byte pair encoding
# 분적을 생성 사능하고 패턴을 만들어서 불용어 처리를 할 수 있다
# * 불용어 처리란? 토큰으로 삼아도 아무 제공을 못하는 애들, 의미전달을 못하는 애들

In [37]:
from nltk.corpus import stopwords

In [41]:
stopwords.open(stopwords.fileids()[8]).read().split()

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [42]:
# Normalization
# 일반화 정규화
# 앞의 과정들은 feature 추출
# 이때 normalization은 각 단어들을 하나의 format으로 만들어주는 과정이다
# 이때, - 혹은 ' 과 같은 애들로 인해 단어의 의미가 변절될 우려가 생긴다

In [44]:
stop = stopwords.open('english').read()

In [48]:
from string import punctuation

# normalization 단계
# 1. 대소문자 일치
# 2. 문장 기호 어떻게 처리할지 = 여기서는 삭제 해보자
# 3. 불용어 처리 => 선택적으로 그리고 전략적으로
# 도메인에 맞는 단어와 내용에 맞게 불용어 처리를 해야한다

d = "To be or not to be."
r = []
for t in re.sub(f'[{punctuation}]', '',  d.lower()).split():
    if t not in stop:
        r.append(t)
r

[]